In [1]:
%load_ext watermark
%watermark -a 'Sebastian Raschka' -v -p torch,pytorch_lightning

Author: Sebastian Raschka

Python implementation: CPython
Python version       : 3.8.12
IPython version      : 8.0.1

torch            : 1.10.1
pytorch_lightning: 1.5.10



The little caveat is that it currently only supports PyTorch Lightning < 1.6.0. So, if you have 1.6.0 or newer, you'd need to downgrade:

In [2]:
#!pip install pytorch-lightning==1.5.10

# K-Fold Cross-Validation Example -- Convolutional Neural Network on MNIST

This notebook shows how to apply k-fold cross-validation to deep neural networks using PyTorch Lightning and [`pl_cross`](https://github.com/SkafteNicki/pl_cross).

This notebook is based on a basic PyTorch Lightning notebook at [./baseline-cnn-mnist.ipynb](baseline-cnn-mnist.ipynb). The changes are annotated and/or highlighted.

## Imports

In [3]:
import os
import time

import numpy as np
import torch
import torch.nn.functional as F

if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## Settings

In [4]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
BATCH_SIZE = 128

# Architecture
NUM_CLASSES = 10

## NEW: Install `pl_cross` Library

Uncomment the following line if you don't have Git Python installed yet.

In [5]:
#!pip install gitpython

In [6]:
import os

from git import Repo

if not os.path.exists("pl_cross"):
    Repo.clone_from("https://github.com/SkafteNicki/pl_cross.git", "pl_cross")

Uncomment the following line to install the cloned repo.

In [7]:
# !cd pl_cross; python setup.py installl; cd ..

## Dataset

In [8]:
if not os.path.exists("mnist-pngs"):
    Repo.clone_from("https://github.com/rasbt/mnist-pngs", "mnist-pngs")

In [9]:
from torchvision import transforms

data_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize(32),
            transforms.RandomCrop((28, 28)),
            transforms.ToTensor(),
            # normalize images to [-1, 1] range
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
    "test": transforms.Compose(
        [
            transforms.Resize(32),
            transforms.CenterCrop((28, 28)),
            transforms.ToTensor(),
            # normalize images to [-1, 1] range
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    ),
}

In [10]:
from torch.utils.data.dataset import random_split
from torchvision.datasets import ImageFolder

train_dset = ImageFolder(root="mnist-pngs/train", transform=data_transforms["train"])

train_dset, valid_dset = random_split(train_dset, lengths=[55000, 5000])

test_dset = ImageFolder(root="mnist-pngs/test", transform=data_transforms["test"])

In [11]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_dset,
    batch_size=BATCH_SIZE,
    drop_last=True,
    num_workers=4,
    shuffle=True,
)

valid_loader = DataLoader(
    dataset=valid_dset,
    batch_size=BATCH_SIZE,
    drop_last=False,
    num_workers=4,
    shuffle=False,
)

test_loader = DataLoader(
    dataset=test_dset,
    batch_size=BATCH_SIZE,
    drop_last=False,
    num_workers=4,
    shuffle=False,
)

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
import torchvision.utils as vutils

real_batch = next(iter(train_loader))
plt.figure(figsize=(8, 8))
plt.axis("off")
plt.title("Training images")
plt.imshow(
    np.transpose(
        vutils.make_grid(real_batch[0][:64], padding=2, normalize=True), (1, 2, 0)
    )
)
plt.show()

## Model

The model setup is exactly as it is before, except that the current k-fold code won't be using `validation_step`. We can still leave it in the `LightningModule` as it doesn't hurt either.

In [13]:
import torch


class PyTorchCNN(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        self.num_classes = num_classes
        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(
                in_channels=3,
                out_channels=8,
                kernel_size=(3, 3),
                stride=(1, 1),
                padding=1,
            ),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
            torch.nn.ReLU(),
            torch.nn.Conv2d(
                in_channels=8,
                out_channels=16,
                kernel_size=(3, 3),
                stride=(1, 1),
                padding=1,
            ),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0),
        )

        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(784, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [14]:
import pytorch_lightning as pl
import torchmetrics


# LightningModule that receives a PyTorch model as input
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()

        self.learning_rate = learning_rate
        # The inherited PyTorch module
        self.model = model

        # Save settings and hyperparameters to the log directory
        # but skip the model parameters
        self.save_hyperparameters(ignore=["model"])

        # Set up attributes for computing the accuracy
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    # Defining the forward method is only necessary
    # if you want to use a Trainer's .predict() method (optional)
    def forward(self, x):
        return self.model(x)

    # A common forward step to compute the loss and labels
    # this is used for training, validation, and testing below
    def _shared_step(self, batch):
        features, true_labels = batch
        logits = self(features)
        loss = torch.nn.functional.cross_entropy(logits, true_labels)
        predicted_labels = torch.argmax(logits, dim=1)

        return loss, true_labels, predicted_labels

    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("train_loss", loss)

        # To account for Dropout behavior during evaluation
        self.model.eval()
        with torch.no_grad():
            _, true_labels, predicted_labels = self._shared_step(batch)
        self.train_acc.update(predicted_labels, true_labels)
        self.log("train_acc", self.train_acc, on_epoch=True, on_step=False)
        self.model.train()
        return loss  # this is passed to the optimzer for training

    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("valid_loss", loss)
        self.valid_acc(predicted_labels, true_labels)
        self.log(
            "valid_acc", self.valid_acc, on_epoch=True, on_step=False, prog_bar=True
        )

    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.test_acc(predicted_labels, true_labels)
        self.log("test_acc", self.test_acc, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

## Training

NEW: Note that for logging, we need to import a modified version of the logger from the `pl_cross` library. Also, since the k-fold approach doesn't use `validation_step` we can't checkpoint based on teh validation set accuracy and have the `training_accuracy` instead.

In [15]:
# OLD:
# from pytorch_lightning.loggers import CSVLogger
# NEW:
from pl_cross.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint

torch.manual_seed(RANDOM_SEED)
pytorch_model = PyTorchCNN(num_classes=NUM_CLASSES)

lightning_model = LightningModel(model=pytorch_model, learning_rate=LEARNING_RATE)


# NEW: Kfold currently doesn't call model.validation_step()
# so the valid_acc tracking does unfortunately not work, and
# we have to change valid_acc to train_acc.

callbacks = [
    ModelCheckpoint(save_top_k=1, mode="max", monitor="train_acc"),  # save top 1 model
    pl.callbacks.progress.TQDMProgressBar(refresh_rate=50),
]

logger = CSVLogger(save_dir="logs/", name="my-kfold-model")

NEW: Instead of using `pl.Trainer`, we use the `Trainer` from the `pl_cross` library, which lets us specify the number of folds as shown below:

In [16]:
import time

# NEW:
from pl_cross import Trainer

trainer = Trainer(
    num_folds=5,
    shuffle=True,
    # stratified=True,
    max_epochs=NUM_EPOCHS,
    callbacks=callbacks,
    accelerator="auto",  # Uses GPUs or TPUs if available
    devices="auto",  # Uses all available GPUs/TPUs if applicable
    logger=logger,
    log_every_n_steps=100,
)

start_time = time.time()

# NEW
kfold_results = trainer.cross_validate(
    model=lightning_model,
    train_dataloader=train_loader,
)

runtime = (time.time() - start_time) / 60
print(f"Training took {runtime:.2f} min in total.")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | model     | PyTorchCNN | 103 K 
1 | train_acc | Accuracy   | 0     
2 | valid_acc | Accuracy   | 0     
3 | test_acc  | Accuracy   | 0     
-----------------------------------------
103 K     Trainable params
0         Non-trainable params
103 K     Total params
0.413     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/jovyan/conda/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:489: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
Starting fold 1/5


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

Starting fold 5/5


Training: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

{'train_loss_mean': tensor(0.0157), 'train_loss_std': tensor(0.0082), 'train_loss_raw': tensor([0.0139, 0.0143, 0.0181, 0.0046, 0.0274]), 'train_acc_mean': tensor(0.9892), 'train_acc_std': tensor(0.0006), 'train_acc_raw': tensor([0.9883, 0.9892, 0.9890, 0.9896, 0.9898]), 'test_acc_mean': tensor(0.9850), 'test_acc_std': tensor(0.0023), 'test_acc_raw': tensor([0.9818, 0.9836, 0.9868, 0.9875, 0.9855])}
Training took 6.85 min in total.


NEW: We can now look at the different test-fold accuracies, which are returned by `trainer.cross_validate`:

In [17]:
print(kfold_results)

{'train_loss_mean': tensor(0.0157), 'train_loss_std': tensor(0.0082), 'train_loss_raw': tensor([0.0139, 0.0143, 0.0181, 0.0046, 0.0274]), 'train_acc_mean': tensor(0.9892), 'train_acc_std': tensor(0.0006), 'train_acc_raw': tensor([0.9883, 0.9892, 0.9890, 0.9896, 0.9898]), 'test_acc_mean': tensor(0.9850), 'test_acc_std': tensor(0.0023), 'test_acc_raw': tensor([0.9818, 0.9836, 0.9868, 0.9875, 0.9855])}


## Evaluation

NEW: We could consider the cross-validation accuracy for model selection and once we are happy with the hyperparameter settings, we could use those to train a new model using the regular trainer approach as in [./baseline-cnn-mnist.ipynb](baseline-cnn-mnist.ipynb) (except that we use the best hyperparameters from k-fold cross-validation). 

Alternatively, we can create an ensemble model from the *k* models as follows:

In [18]:
# ensemble_model = trainer.create_ensemble(model=lightning_model)

In [19]:
# trainer.test(model=ensemble_model, dataloaders=test_loader)

Note: for some reason, the `trainer.create_ensemble` is complaining about the checkpoint files, it is potentially a bug. Will update once it is fixed or I know more :).

In [20]:
%watermark -iv

numpy            : 1.22.2
torchvision      : 0.11.2
torch            : 1.10.1
pytorch_lightning: 1.5.10
torchmetrics     : 0.6.2
matplotlib       : 3.3.4

